<a id='top'></a>
# Random Forests

### May 2017

<br>
This is a study for a blog post to appear on [Data Simple](http://luisvalesilva.com/datasimple). It will focus on the theory and scikit-learn implementation of the **Random Forest** machine learning (ML) algorithm.
<br><br>

## Contents

1. [Introduction](#intro)
1. [Decision Trees](#trees)
1. [Ensemble Learning](#ensemble)

<br><br>

<a id='intro'></a>
# 1. Introduction

Random Forests are a popular example of an [Ensemble Learning](https://en.wikipedia.org/wiki/Ensemble_learning) method. Ensemble Learning consists on combining multiple ML models in order to achieve higher predictive performance than could be obtained using either of the individual models alone.

Ensemble Learning methods work best when the models are as diverse as possible, providing complementary predictive performance. One way to achieve this is to combine models built using different ML algorithms, to try to combine models making errors on different training set instances. An alternative approach is to use the same ML algorithm, but to train each model on a different random subset of the training data set. This is the appproach taken by Random Forests: they combine multiple Decision Tree models trained on different subsets of the training data. The next section provides an overview of the Decision Tree ML algorithm, which is at the core of Random Forests. More on Ensemble Learning theory later.

<a id='trees'></a>
# 2. Decision Trees

<a id='ensemble'></a>
# 3. Ensemble Learning

[Back to top](#top)